In [2]:
import requests, json, os
import pandas as pd

In [3]:
fpl_url = 'https://fantasy.premierleague.com/api/'

In [17]:
# load player data
players = pd.json_normalize(
    requests.get(fpl_url + 'bootstrap-static/').json()['elements']
).drop(
    ['chance_of_playing_next_round',
     'chance_of_playing_this_round',
     'code',
     'cost_change_event',
     'cost_change_event_fall',
     'cost_change_start',
     'cost_change_start_fall',
     'ep_next',
     'ep_this',
     'event_points',
     'form',
     'news',
     'news_added',
     'photo',
     'selected_by_percent',
     'special',
     'squad_number',
     'status',
     'team_code',
     'transfers_in',
     'transfers_in_event',
     'transfers_out',
     'transfers_out_event',
     'value_form',
     'value_season'], axis=1
)

players.head()

,dreamteam_count,element_type,first_name,id,in_dreamteam,now_cost,points_per_game,second_name,team,total_points,...,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type
0,0,2,Shkodran,1,False,51,2.9,Mustafi,1,43,...,155.0,47.9,257,100,352,122,234,58,313,106
1,0,2,Héctor,2,False,52,2.9,Bellerín,1,44,...,103.0,37.0,322,124,301,95,275,81,339,122
2,0,2,Sead,3,False,52,2.1,Kolasinac,1,55,...,81.0,53.0,264,104,215,52,305,98,293,96
3,1,2,Ainsley,4,False,45,2.0,Maitland-Niles,1,41,...,58.0,53.6,241,94,216,53,337,119,287,94
4,1,2,Sokratis,5,False,48,3.0,Papastathopoulos,1,57,...,110.0,58.5,166,62,365,130,266,76,275,88


In [19]:
# load team data
teams = pd.json_normalize(
    requests.get(fpl_url + 'bootstrap-static/').json()['teams']
).drop(
    ['code',
     'form',
     'pulse_id',
     'draw',
     'loss',
     'played',
     'points',
     'position',
     'team_division',
     'unavailable',
     'win'], axis=1
)

teams.head()

,id,name,short_name,strength,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away
0,1,Arsenal,ARS,4,1180,1240,1170,1170,1150,1200
1,2,Aston Villa,AVL,2,1020,1050,970,980,1000,1040
2,3,Bournemouth,BOU,2,1020,1020,990,1030,1000,1050
3,4,Brighton,BHA,2,1050,1010,1100,1100,1040,1030
4,5,Burnley,BUR,3,1110,1180,1130,1070,970,1110


In [20]:
# load position information
positions = pd.json_normalize(
    requests.get(fpl_url + 'bootstrap-static/').json()['element_types']
)[
    ['id',
     'plural_name',
     'plural_name_short',
     'singular_name'
    ]
]

positions.head()

,id,plural_name,plural_name_short,singular_name
0,1,Goalkeepers,GKP,Goalkeeper
1,2,Defenders,DEF,Defender
2,3,Midfielders,MID,Midfielder
3,4,Forwards,FWD,Forward


In [42]:
def get_gameweek_history(player_id):
    
    df = pd.json_normalize(
        requests.get(fpl_url + 'element-summary/' + str(player_id) + '/').json()['history']
    )
    
    return df
    
pd.merge(left=players[['id', 'web_name']],
         right=get_gameweek_history(1),
         left_on='id',
         right_on='element').head()

,id,web_name,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,...,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,Mustafi,1,10,13,0,False,2019-08-11T13:00:00Z,0,1,...,0,0.0,0.0,0.0,0.0,55,0,33117,0,0
1,1,Mustafi,1,11,5,0,True,2019-08-17T11:30:00Z,2,1,...,0,0.0,0.0,0.0,0.0,55,-5280,36709,2868,8148
2,1,Mustafi,1,24,10,0,False,2019-08-24T16:30:00Z,3,1,...,0,0.0,0.0,0.0,0.0,54,-6882,30975,534,7416
3,1,Mustafi,1,31,17,0,True,2019-09-01T15:30:00Z,2,2,...,0,0.0,0.0,0.0,0.0,54,-3872,28096,346,4218
4,1,Mustafi,1,49,18,0,False,2019-09-15T15:30:00Z,2,2,...,0,0.0,0.0,0.0,0.0,53,-2073,26902,581,2654


In [40]:
def get_season_history(player_id):
    
    df = pd.json_normalize(
        requests.get(fpl_url + 'element-summary/' + str(player_id) + '/').json()['history_past']
    )
    
    df['id'] = player_id
    
    return df

pd.merge(left=players[['id', 'web_name']],
         right=get_season_history(1),
         on='id')

,id,web_name,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,...,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
0,1,Mustafi,2009/10,69140,45,45,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,1,Mustafi,2010/11,69140,40,40,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,1,Mustafi,2011/12,69140,50,50,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
3,1,Mustafi,2012/13,69140,40,40,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
4,1,Mustafi,2016/17,69140,60,58,90,2274,2,2,...,0,11,0,0,8,513,598.2,159.7,364.0,112.4
5,1,Mustafi,2017/18,69140,55,54,109,2272,3,1,...,0,6,0,0,13,610,836.6,129.9,202.0,117.0
6,1,Mustafi,2018/19,69140,55,54,80,2611,2,0,...,0,9,0,0,5,475,718.6,106.0,252.0,107.8
7,1,Mustafi,2019/20,69140,55,51,43,1205,0,2,...,0,2,0,0,2,242,277.2,45.5,155.0,47.9
